In [2]:
import os
import sys
sys.path.append("../dnn")
sys.path.append("../utils")

import torch
from torch.autograd import Variable

from ipywidgets import interact, fixed
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns; sns.set()

from dnn_md import DNNMultidecoder
from hao_data import read_next_utt

In [4]:
# Set up environment variables for the model we want to examine
# NOT necessarily the current environment variables!!

feat_dim=80
left_splice=5
right_splice=5

optimizer="Adam"
learning_rate=0.0001
epochs=100
batch_size=256

enc_layers=[1024]
latent_dim=512
dec_layers=[1024]
activation="SELU"

enc_layers_delim="_" + "_".join(map(str, enc_layers))
if len(enc_layers) == 0:
     # need to recreate bash join behavior with empty array
    enc_layers_delim="_"
dec_layers_delim="_" + "_".join(map(str, dec_layers))
if len(dec_layers) == 0:
     # need to recreate bash join behavior with empty array
    dec_layers_delim="_"

debug_model = False
debug_str = "true" if debug_model else "false"
expt_name = "ENC_%s_LATENT_%s_DEC_%s_ACT_%s/OPT_%s_LR_%s_EPOCHS_%d_BATCH_%d_DEBUG_%s" % (enc_layers_delim, latent_dim, dec_layers_delim, activation, optimizer, str(learning_rate), epochs, batch_size, debug_str)

input_dim = (left_splice + right_splice + 1) * feat_dim
print("Using experiment %s" % expt_name)

Using experiment ENC__1024_LATENT_512_DEC__1024_ACT_SELU/OPT_Adam_LR_0.0001_EPOCHS_100_BATCH_256_DEBUG_false


In [9]:
# TODO
# Load per-utterance latent var values
model_dir = "/data/sls/scratch/atitus5/meng/models"

latent_scp_path = "%s/ae_noreg/%s/latent.scp" % (model_dir, expt_name_noreg)
print("Using scp %s" % latent_scp_path)

Using noreg scp /data/sls/scratch/atitus5/timit-latent-regularization/latent/ae_noreg/ENC__2048_LATENT_512_DEC__2048_ACT_SELU/OPT_Adam_LR_0.0001_EPOCHS_250_BATCH_256/latent.scp
Using l2 scp /data/sls/scratch/atitus5/timit-latent-regularization/latent/ae_l2/ENC__2048_LATENT_512_DEC__2048_ACT_SELU/OPT_Adam_LR_0.0001_EPOCHS_250_BATCH_256/L2_LAMBDA_0.5/latent.scp


In [10]:
utt_ids = []

latent_vals_noreg = []
with open(latent_scp_path_noreg, 'r') as latent_scp:
    for scp_line in latent_scp:
        utt_id, latent, ark = read_next_utt(scp_line)
        latent_vals_noreg.append(latent)
        utt_ids.append(utt_id)
print("Read %d noreg utterances" % len(latent_vals_noreg))

latent_vals_l2 = []
with open(latent_scp_path_l2, 'r') as latent_scp:
    utt_id_idx = 0
    for scp_line in latent_scp:
        utt_id, latent, ark = read_next_utt(scp_line)
        latent_vals_l2.append(latent)
        assert(utt_id == utt_ids[utt_id_idx])
        utt_id_idx += 1
print("Read %d l2 utterances" % len(latent_vals_l2))

Read 192 noreg utterances
Read 192 l2 utterances


In [12]:
def latentHeatmapComp(utt_idx):
    x_skip = int(latent_vals_noreg[utt_idx].shape[0] / 10.0)
    y_skip = int(latent_vals_noreg[utt_idx].shape[1] / 10.0)
    
    ax = sns.heatmap(np.transpose(latent_vals_noreg[utt_idx]), cmap="coolwarm", xticklabels=x_skip, yticklabels=y_skip)
    ax.set_title("Latent vars (noreg)")
    plt.xlabel("Frame index")
    plt.ylabel("Latent variable index")
    plt.savefig("latent_ae_noreg.png")
    plt.show()
    
    ax = sns.heatmap(np.transpose(latent_vals_l2[utt_idx]), cmap="coolwarm", xticklabels=x_skip, yticklabels=y_skip)
    ax.set_title("Latent vars (l2)")
    plt.xlabel("Frame index")
    plt.ylabel("Latent variable index")
    plt.savefig("latent_ae_l2.png")
    plt.show()

interact(latentHeatmapComp, utt_idx=(0, len(utt_ids) - 1, 1))

<function __main__.latentHeatmapComp>